## Output has been removed from this notebook to reduce file sizes in the repo

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import pandana as pdna
from pandana.loaders import osm
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## Download OpenStreetMap restaurants for a good part of the Bay Area

###### Note: used http://boundingbox.klokantech.com/ to get the bounding box

In [ ]:
# Bounding box from link above
tmp = [-122.8662, 37.1373, -121.4798, 38.2158]

# Reordered for Pandana functions
bbox = [tmp[1], tmp[0], tmp[3], tmp[2]]

poi_df = osm.node_query(*bbox, tags='amenity=restaurant')
x, y = poi_df['lon'], poi_df['lat']

## Get previously stored OpenStreetMap networks for Bay Area

Download the data here: https://s3-us-west-1.amazonaws.com/synthpop-data2/pandana/osm_bayarea.h5

In [ ]:
store = pd.HDFStore('data/osm_bayarea.h5', "r")
nodes = store.nodes
edges = store.edges
print(nodes.head(3))
print(edges.head(3))

## Initialize and preprocess the network

In [ ]:
net=pdna.Network(nodes.x, 
                 nodes.y, 
                 edges["from"], 
                 edges["to"],
                 edges[["weight"]])
net.precompute(3000)

## Nearest *point-of-interest* queries

In [ ]:
net.set_pois("restaurants", 2000, 10, x, y)

In [ ]:
a = net.nearest_pois(2000, "restaurants", num_pois=10)
print(a.head(1))

## Here's a map of the distance to the nearest restaurant

In [ ]:
fig_kwargs = {'figsize': [20, 20]}
bmap_kwargs = {'suppress_ticks': False, 'resolution': 'h', 'epsg': '26943'}
plot_kwargs = {'cmap': 'BrBG', 's': 8, 'edgecolor': 'none'}

sf_tmp = [-122.524338, 37.707794, -122.34993, 37.834192]
sf_bbox = [sf_tmp[1], sf_tmp[0], sf_tmp[3], sf_tmp[2]]

net.plot(a[1], bbox=sf_bbox, 
         fig_kwargs=fig_kwargs, bmap_kwargs=bmap_kwargs, plot_kwargs=plot_kwargs)

## Here's a map of the distance to the 5th nearest restaurant

In [ ]:
net.plot(a[5], bbox=sf_bbox, 
         fig_kwargs=fig_kwargs, bmap_kwargs=bmap_kwargs, plot_kwargs=plot_kwargs)

## Here's a map of the distance to the 10th nearest restaurant

In [ ]:
net.plot(a[10], bbox=sf_bbox, 
         fig_kwargs=fig_kwargs, bmap_kwargs=bmap_kwargs, plot_kwargs=plot_kwargs)

# A similar workflow is used to do general network aggregations

## Relate the x-ys to nodes

In [ ]:
node_ids = net.get_node_ids(x, y)

## Assign the variable (in this case just location) to the network

In [ ]:
net.set(node_ids)

## This is it - run the queries!

In [ ]:
%time s = net.aggregate(500, type="sum", decay="linear")
%time t = net.aggregate(1000, type="sum", decay="linear")
%time u = net.aggregate(2000, type="sum", decay="linear")
%time v = net.aggregate(3000, type="sum", decay="linear")
%time w = net.aggregate(3000, type="count", decay="flat")

## Here's a map of access to restaurants with a 500m radius

In [ ]:
net.plot(s, bbox=sf_bbox, 
         fig_kwargs=fig_kwargs, bmap_kwargs=bmap_kwargs, plot_kwargs=plot_kwargs)

## Or 1000 meters

In [ ]:
net.plot(t, bbox=sf_bbox, 
         fig_kwargs=fig_kwargs, bmap_kwargs=bmap_kwargs, plot_kwargs=plot_kwargs)

## Or 2000 meters radius

In [ ]:
net.plot(u, bbox=sf_bbox, 
         fig_kwargs=fig_kwargs, bmap_kwargs=bmap_kwargs, plot_kwargs=plot_kwargs)

## Or 3000m radius

In [ ]:
net.plot(v, bbox=sf_bbox, 
         fig_kwargs=fig_kwargs, bmap_kwargs=bmap_kwargs, plot_kwargs=plot_kwargs)

## Or the whole Bay Area region

In [ ]:
net.plot(w, bbox=bbox, 
         fig_kwargs=fig_kwargs, bmap_kwargs=bmap_kwargs, plot_kwargs=plot_kwargs)